In [1]:
from torch import Generator, use_deterministic_algorithms, randint, Tensor
from functools import partial
from typing import Tuple, Iterator

from pathlib import Path
from karpathy_series.makemore.data import read_data
from karpathy_series.makemore.encoding.character import CharacterEncoder, CharacterSet, StringEncoder
from karpathy_series.makemore.training.data import TrainingSequencer
from karpathy_series.makemore.training.learning import Learner
from karpathy_series.makemore.models.sequential import MPLNet, NGramNetGenerator
from karpathy_series.makemore.bigrams import NGram

%matplotlib inline
use_deterministic_algorithms(True)
generator = Generator()
seed = 2147483647

In [2]:
data_path = Path("../../data/names.txt")
words = read_data(data_path)
char_set = CharacterSet.from_words(words)
char_encoder = CharacterEncoder.from_charset(char_set)
string_encoder = StringEncoder(char_encoder)

context_size = 4
n_gram_gen = partial(NGram.generate, context_size, char_set.pad) 
ts = TrainingSequencer(string_encoder, char_encoder, n_gram_gen)

embedding_dims = 10
hidden_dims = 200

In [3]:
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

xis_t, yis_t = ts.training_set(words[:n1])
xis_d, yis_d = ts.training_set(words[n1:n2])
xis_v, yis_v = ts.training_set(words[n2:])
print(xis_t.shape, xis_d.shape, xis_v.shape)

g = generator.manual_seed(seed)
def batcher(batch: int, xs: Tensor, ys: Tensor) -> Iterator[Tuple[Tensor, Tensor]]:
    length = xs.shape[0]
    cover = length // batch
    for _ in range(cover):
        index = randint(0, length, (batch,), generator=g)
        yield xs[index], ys[index]

learning_sequence = partial(batcher, 100, xis_t, yis_t)

torch.Size([182625, 4]) torch.Size([22655, 4]) torch.Size([22866, 4])


In [4]:
g = generator.manual_seed(seed)
mpl_net = MPLNet.init_random_from_size(char_encoder.size, context_size, embedding_dims, hidden_dims, generator=g)
generator = NGramNetGenerator(char_set, string_encoder, char_encoder, mpl_net)

In [13]:
learner = Learner(mpl_net, 0.01)
learner(learning_sequence, epochs=10, report_epochs=1)

Epoch 1 is finished with loss = 2.350008010864258
Epoch 2 is finished with loss = 2.147059440612793
Epoch 3 is finished with loss = 2.352504014968872
Epoch 4 is finished with loss = 2.4430510997772217
Epoch 5 is finished with loss = 2.1677920818328857
Epoch 6 is finished with loss = 1.9552032947540283
Epoch 7 is finished with loss = 2.16409969329834
Epoch 8 is finished with loss = 2.164581060409546
Epoch 9 is finished with loss = 2.3554062843322754
Epoch 10 is finished with loss = 2.0410265922546387


In [14]:
print(mpl_net.run(xis_v, yis_v))

tensor(2.2019, grad_fn=<NllLossBackward0>)


In [16]:
for k in range(20):
    print(generator())

keensky
anna
kamari
iishaner
brin
baserices
adalyn
mararin
shana
baie
riannuah
adur
manna
elio
xopiyta
kentir
soviah
zora
nyca
avyva
